In [3]:
import numpy as np
import pandas as pd
import os

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from scipy.special import rel_entr
import scipy.stats

In [4]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
! nvidia-smi

Sat Mar 19 22:54:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 26%   34C    P8    19W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 25%   

# Company數量

In [6]:
from pymongo import MongoClient
import pymongo

In [7]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.patent_de
    db_us = db.patent_us
    db_cn = db.patent_cn
except errors.ConnectionFailure as err:
    print(err)

In [8]:
data_assignee = db_de.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [9]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
data_assignee = db_cn.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [11]:
data_assignee = db_us.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [12]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [13]:
# 取總數前15家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)

,company,count
45,Qualcomm Inc,8491
49,Samsung Electronics Co Ltd,6337
28,Intel Corp,3236
61,Telefonaktiebolaget LM Ericsson AB,2488
24,Huawei Technologies Co Ltd,2461
33,LG Electronics Inc,1931
30,International Business Machines Corp,1591
2,Apple Inc,1104
9,Cisco Technology Inc,1092
36,Microsoft Technology Licensing LLC,963


In [14]:
com_ls = list(final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)["company"])
com_ls

['Qualcomm Inc',
 'Samsung Electronics Co Ltd',
 'Intel Corp',
 'Telefonaktiebolaget LM Ericsson AB',
 'Huawei Technologies Co Ltd',
 'LG Electronics Inc',
 'International Business Machines Corp',
 'Apple Inc',
 'Cisco Technology Inc',
 'Microsoft Technology Licensing LLC',
 'Micron Technology Inc',
 'Sony Corp',
 'Nokia Technologies Oy',
 'Shenzhen Shenglu IoT Communication Technology Co Ltd',
 'NTT Docomo Inc']

['Qualcomm Inc',
 'Samsung Electronics Co Ltd',
 'Intel Corp',
 'Telefonaktiebolaget LM Ericsson AB',
 'Huawei Technologies Co Ltd',
 'LG Electronics Inc',
 'International Business Machines Corp',
 'Apple Inc',
 'Cisco Technology Inc',
 'Microsoft Technology Licensing LLC',
 'Micron Technology Inc',
 'Sony Corp',
 'Nokia Technologies Oy',
 'Shenzhen Shenglu IoT Communication Technology Co Ltd',
 'NTT Docomo Inc']

# Data

In [15]:
ccontry = pd.read_csv("./final_data/company_country_all.csv")

In [16]:
ccpc = pd.read_csv("./final_data/company_cpc_all.csv")

In [17]:
ccontry[ccontry.company.isin(com_ls)]

,company,AP,AR,AT,AU,BE,BG,BR,CA,CH,...,SI,SM,TN,TR,TW,UA,US,UY,WO,ZA
716,Apple Inc,0.0,0.0,0.0,8.0,0.0,0.0,18.0,2.0,0.0,...,0.0,0.0,0.0,0.0,62.0,0.0,1045.0,0.0,1013.0,0.0
2958,Cisco Technology Inc,0.0,0.0,0.0,13.0,0.0,0.0,0.0,54.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1160.0,0.0,326.0,0.0
6392,Huawei Technologies Co Ltd,0.0,2.0,0.0,75.0,0.0,0.0,204.0,81.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1341.0,0.0,2117.0,4.0
6965,Intel Corp,0.0,0.0,0.0,9.0,0.0,0.0,60.0,4.0,0.0,...,0.0,0.0,0.0,0.0,167.0,0.0,3254.0,0.0,1546.0,0.0
7007,International Business Machines Corp,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1687.0,0.0,142.0,2.0
8111,LG Electronics Inc,0.0,1.0,0.0,31.0,0.0,0.0,25.0,16.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,1859.0,0.0,1294.0,1.0
8757,Micron Technology Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,46.0,0.0,709.0,0.0,361.0,0.0
8759,Microsoft Technology Licensing LLC,0.0,0.0,0.0,17.0,0.0,0.0,14.0,17.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,956.0,0.0,793.0,8.0
9032,NTT Docomo Inc,0.0,4.0,0.0,41.0,0.0,0.0,89.0,64.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,470.0,0.0,670.0,4.0
9723,Nokia Technologies Oy,0.0,4.0,0.0,25.0,0.0,0.0,26.0,25.0,0.0,...,0.0,0.0,0.0,0.0,21.0,0.0,710.0,0.0,592.0,4.0


# Similarity計算

In [18]:
# 換成機率(?)
# 比例vs機率(?)

In [19]:
ccontry.set_index("company", inplace=True)
ccontry2 = ccontry.apply(lambda x: x/sum(x), axis=1) #比例

In [22]:
P = ccontry2[ccontry2.index==com_ls[0]].values.tolist()[0][1:]
Q = ccontry2[ccontry2.index==com_ls[1]].values.tolist()[0][1:]

In [23]:
print(P)
print(Q)

[0.00039672419167445944, 0.0, 0.014962169514579614, 0.0, 0.0, 0.037632123324548725, 0.01507551928362946, 0.0, 0.0016152342089602993, 0.1511235795857066, 0.0018419337470599903, 0.0, 0.0, 0.0, 0.0, 5.667488452492278e-05, 0.00036838674941199807, 0.0, 0.00022669953809969113, 5.667488452492278e-05, 0.13766329451103743, 0.00467567797330613, 0.0, 0.0, 5.667488452492278e-05, 0.0, 0.0, 0.0003117118648870753, 5.667488452492278e-05, 0.001671909093485222, 0.0, 0.0010768228059735328, 0.0002833744226246139, 0.0, 0.0, 0.037632123324548725, 0.04582164413840007, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0009634730369236873, 0.0, 0.0, 0.0, 0.00014168721131230694, 0.0, 0.0012751849018107625, 0.00011334976904984556, 0.00014168721131230694, 0.0, 5.667488452492278e-05, 0.0013601972285981468, 0.00011334976904984556, 0.0, 0.025163648729065714, 0.0003400493071495367, 0.0, 0.00022669953809969113, 0.0, 0.06101051319107937, 0.0, 0.2364759556802403, 0.0, 0.21953016520728838, 0.00048173651846184363]
[0.0, 0.0, 0.00618062563067608

In [18]:
sum(rel_entr(P, Q))

inf

In [19]:
rel_entr(P, Q)

array([ 0.00000000e+00,             inf,  0.00000000e+00,  6.75134432e+02,
        0.00000000e+00,  0.00000000e+00,  6.63230391e+03,  1.46315054e+03,
        0.00000000e+00,             inf,  1.04866471e+03,             inf,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.02299906e+01,             inf,  0.00000000e+00,             inf,
                   inf,  1.86831586e+03,  1.58863773e+02,  0.00000000e+00,
        0.00000000e+00, -6.86797441e+00,  0.00000000e+00,  0.00000000e+00,
                   inf,             inf,  1.99679026e+02,  0.00000000e+00,
                   inf,  1.20397280e+01,  0.00000000e+00,  0.00000000e+00,
        1.94496434e+03, -1.89562186e+03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.51906222e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,             inf,
        0.00000000e+00,  7.77249427e+01,  1.15072829e+00,  4.58145366e+00,
        0.00000000e+00,  

In [57]:
KL = scipy.stats.entropy(P, Q) 
print(KL)

inf


In [58]:
x = [np.random.randint(1, 11) for i in range(10)]
print(x)
print(np.sum(x))

[8, 7, 2, 2, 6, 2, 10, 1, 5, 5]
48


# PCA

In [ ]:
pca=PCA(n_components=2)
newData=pca.fit_transform(Coke1)
print newData[:4]
x1=[n[0] for n in newData]
x2=[n[1] for n in newData]

In [ ]:
pca = PCA(n_components=2, whiten=True)  # TSNE(n_components=2, n_iter=3000, verbose=2)
pca.fit(np.vstack([src_embeddings, tgt_embeddings]))
print('Variance explained: %.2f' % pca.explained_variance_ratio_.sum())

In [ ]:
def plot_similar_word(src_words, src_word2id, src_emb, tgt_words, tgt_word2id, tgt_emb, pca):

    Y = []
    word_labels = []
    for sw in src_words:
        Y.append(src_emb[src_word2id[sw]])
        word_labels.append(sw)
    for tw in tgt_words:
        Y.append(tgt_emb[tgt_word2id[tw]])
        word_labels.append(tw)

    # find tsne coords for 2 dimensions
    Y = pca.transform(Y)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]

    # display scatter plot
    plt.figure(figsize=(10, 8), dpi=80)
    plt.scatter(x_coords, y_coords, marker='x')

    for k, (label, x, y) in enumerate(zip(word_labels, x_coords, y_coords)):
        color = 'blue' if k < len(src_words) else 'red'  # src words in blue / tgt words in red
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontsize=19,
                     color=color, weight='bold')

    plt.xlim(x_coords.min() - 0.2, x_coords.max() + 0.2)
    plt.ylim(y_coords.min() - 0.2, y_coords.max() + 0.2)
    plt.title('Visualization of the multilingual word embedding space')

    plt.show()

In [ ]:
# get 5 random input words
src_words = ['university', 'love', 'history', 'tennis', 'research', 'conference']
tgt_words = ['universidad', 'amor', 'historia', u'tenis',  u'investigación', 'conferencia']

# assert words in dictionaries
for sw in src_words:
    assert sw in src_word2id, '"%s" not in source dictionary' % sw
for tw in tgt_words:
    assert tw in tgt_word2id, '"%s" not in target dictionary' % sw

plot_similar_word(src_words, src_word2id, src_embeddings, tgt_words, tgt_word2id, tgt_embeddings, pca)